In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

import findspark

findspark.init()

conf = SparkConf().set("spark.ui.port", 1111)
sc = SparkContext("local[2]", appName="WordCountStreaming", conf=conf)
sc


In [ ]:
# in windows command line
# ncat -lk 9999

In [ ]:
ssc = StreamingContext(sc, 10)
lines = ssc.socketTextStream("localhost", 9999)

In [ ]:
# Define function to preprocess text (make lowercase and remove punctuation)
import re


def preprocess_text(text: str) -> str:
    """
    Preprocesses text by converting to lowercase and removing punctuation.
    """
    return re.sub(r"[^\w\s]", "", text.strip().lower())

preprocess_text("He::lLo")


In [ ]:
word_to_count = "hello"

counts = (
    lines.flatMap(lambda lines: lines.split(" "))
    .map(lambda word: (preprocess_text(word), 1))
    .filter(lambda word: word[0] == word_to_count)
    .reduceByKey(lambda a, b: a + b)
)


In [ ]:
counts.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()


In [ ]:
ssc.stop()
sc.stop()